
## dataset link
```
https://drive.google.com/file/d/1OKT4Q5fZ4rDBTY_bDwLSDg5-ZF1BcBeZ/view?usp=sharing
```



## Trying FL

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten,Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
import collections
from matplotlib import pyplot as plt


import os
import shutil 
import math
import random
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


[percpu.cc : 552] RAW: rseq syscall failed with errno 1


In [2]:
# Define a function for federated averaging
def federated_averaging(client_model, centralized_model):
    client_weights = client_model.get_weights()
    central_weights = centralized_model.get_weights()
    for i in range(len(central_weights)):
        central_weights[i] = (central_weights[i] + client_weights[i])/2
    centralized_model.set_weights(central_weights)
    return centralized_model

In [3]:
#create clients
#clients = create_clients(X_train, y_train, num_clients=3, initial='client')

clients = []
clients.append('1')
clients.append('2')
clients.append('3')
#process and batch the training data for each client
clients_batched = dict()
for client_name in clients:
    # client_name format: clients_1
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2 , rescale = 1./255 , horizontal_flip=True)
    train_data = train_datagen.flow_from_directory(directory= "/kaggle/input/rakib-cervical/client test/client"+client_name, target_size=(66, 66), batch_size=10, class_mode = 'categorical')
    # clients_batched[client_name] = batch_data(data)
    clients_batched[client_name] = train_data
    
#process and batch the test set  
#test_data = train_datagen.flow_from_directory(directory= "/content/test"+client_name, target_size=(66, 66), batch_size=100, class_mode = 'categorical')
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_data = test_datagen.flow_from_directory(directory= "/kaggle/input/rakib-cervical/client test/test", target_size=(66, 66), batch_size=10, class_mode = 'categorical')

comms_round = 20

#create optimizer
lr = 0.001
loss='categorical_crossentropy'
metrics = ['accuracy'] # f1_score, precision_score
optimizer = tf.keras.optimizers.legacy.SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               ) 
# optimizer = tf.train.AdamOptimizer(learning_rate=lr, name='Adam-op').minimize(loss)

Found 1120 images belonging to 5 classes.
Found 1125 images belonging to 5 classes.
Found 1119 images belonging to 5 classes.
Found 685 images belonging to 5 classes.


In [4]:
# Model architecture: MLP
class SimpleMLP:
    @staticmethod
    def build():
        model = Sequential() 
        model.add(Conv2D(filters=16, kernel_size= (3,3), activation= 'relu', input_shape=(66,66,3)) )
        model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        
        model.add(BatchNormalization())
        
        model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(rate=0.25))
        model.add(Flatten())
        model.add(Dense(units=256, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dropout(rate=0.25))
        model.add(Dense(units=5, activation='sigmoid'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']  )
        #model.summary()
        
        return model

In [5]:
# # Model architecture: MLP
# class SimpleMLP:
#     @staticmethod
#     def build():

#         model=Sequential()


#         #covolution layer
#         model.add(Conv2D(32,(3,3),activation='relu',input_shape=(66,66,3)))
#         #pooling layer
#         model.add(MaxPooling2D(2,2))
#         model.add(BatchNormalization())
#         #covolution layer
#         model.add(Conv2D(32,(3,3),activation='relu'))
#         #pooling layer
#         model.add(MaxPooling2D(2,2))
#         model.add(BatchNormalization())
#         #covolution layer
#         model.add(Conv2D(64,(3,3),activation='relu'))
#         #pooling layer
#         model.add(MaxPooling2D(2,2))
#         model.add(BatchNormalization())
#         #covolution layer
#         model.add(Conv2D(64,(3,3),activation='relu'))
#         #pooling layer
#         model.add(MaxPooling2D(2,2))
#         model.add(BatchNormalization())
#         #i/p layer
#         model.add(Flatten())
#         #o/p layer
#         model.add(Dense(3,activation='softmax'))
#         model.summary()
        
#         model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
        
#         return model

In [6]:
# class SimpleMLP:
#     @staticmethod
#     def build():
#         # Load the ResNet50 model without the top layer
#         base_model = ResNet50(include_top=False, input_shape=(66, 66, 3))

#         # Add new classification layers on top of the pre-trained model
#         x = base_model.output
#         x = GlobalAveragePooling2D()(x)
#         x = Dense(1024, activation='relu')(x)
#         predictions = Dense(5, activation='softmax')(x)

#         # Combine the base model and new layers to create the final model
#         model = Model(inputs=base_model.input, outputs=predictions)

#         # Freeze the pre-trained layers of the base model
#         for layer in base_model.layers:
#             layer.trainable = False

#         # Compile the model with appropriate loss and optimizer
#         model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#         return model

## Train local models

In [7]:
# todo : 
    #     shob local model tader weight update gula send korbe and then ei list e append korbe (scaled_local_weight_list.append(scaled_weights))
         
    #     #clear session to free memory after each communication round
    #     K.clear_session()
        
    # #to get the average over all the local model, we simply take the sum of the scaled weights
    # average_weights = sum_scaled_weights(scaled_local_weight_list)

    # return average_weights

In [8]:
def train_local_models(current_client, global_weights,global_model):
     #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    #random.shuffle(client_names)

    smlp_temp = SimpleMLP()
    temp_model = smlp_global.build()
    #temp_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    temp_model.set_weights(global_weights)
    
        
    #loop through each client and create new local model
    for client in client_names:
        print("Common Round no: ",end=" ")
        print(current_client+1,end="   ")
        print("Child no: ",end=" ")
        print(client)
        smlp_local = SimpleMLP()
        local_model = smlp_local.build()
        local_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        local_model.fit(clients_batched[client],steps_per_epoch=60, epochs=30, verbose=1)

        temp_model = federated_averaging(local_model,temp_model)

        K.clear_session()

    return temp_model

## Aggregation of Global Model


In [9]:
def aggregate(global_model):
    #commence global training loop
    for comm_round in range(comms_round):
        print("common round: ",end="")
        print(comm_round+1)
        # step_0 : get the global model's weights - will serve as the initial weights for all local models
        global_weights = global_model.get_weights()
        
        # step_1 : call the function for local models training
        global_model = train_local_models(comm_round, global_weights,global_model)
    return global_model

## Initialize the global Model

In [10]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build()
global_model = aggregate(global_model)
  

common round: 1
Common Round no:  1   Child no:  1
Epoch 1/30
60/60 [==============================] - 4s 52ms/step - loss: 1.3129 - accuracy: 0.4183
Epoch 2/30
60/60 [==============================] - 2s 40ms/step - loss: 0.8860 - accuracy: 0.6567
Epoch 3/30
60/60 [==============================] - 2s 37ms/step - loss: 0.7423 - accuracy: 0.7067
Epoch 4/30
60/60 [==============================] - 2s 34ms/step - loss: 0.6417 - accuracy: 0.7533
Epoch 5/30
60/60 [==============================] - 2s 32ms/step - loss: 0.5499 - accuracy: 0.8100
Epoch 6/30
60/60 [==============================] - 2s 32ms/step - loss: 0.5590 - accuracy: 0.8083
Epoch 7/30
60/60 [==============================] - 2s 32ms/step - loss: 0.5368 - accuracy: 0.8350
Epoch 8/30
60/60 [==============================] - 2s 32ms/step - loss: 0.4781 - accuracy: 0.8183
Epoch 9/30
60/60 [==============================] - 2s 33ms/step - loss: 0.4647 - accuracy: 0.8567
Epoch 10/30
60/60 [==============================] - 2s 32

In [11]:
global_model.save('cervical_cancer_best_model.h5')

In [12]:
global_model = load_model("./cervical_cancer_best_model.h5")

In [13]:
# Checking the Accuracy of the Model 
accuracy = global_model.evaluate_generator(generator= test_data)[1] 
print(f"The accuracy of your model is = {accuracy*100} %")

The accuracy of your model is = 87.00730204582214 %
